In [1]:
!pip install antropy
!pip install hurst
!pip install pyentrp

In [13]:
import scipy.io as sio
import pandas as pd
import numpy as np
from scipy.stats import kurtosis,skew
import statistics as st
from antropy import petrosian_fd as pfd 
from antropy import detrended_fluctuation as dfa
from hurst import compute_Hc as he
from pyentrp import entropy

'''
testmeta = pd.read_csv('test_meta.csv')
testsignal = pd.read_csv('test_signal.csv')
'''
testmeta = pd.read_csv('test_meta.csv')
testsignal = pd.read_csv('test_signal.csv')

In [14]:
'''
trainmeta = trainmeta.drop(['MI','height','site','nurse','device', 'STTC', 'HYP', 'CD', 'sub_NORM', 'sub_IMI', 'sub_STTC',
       'sub_NST_', 'sub_LVH', 'sub_LAFB/LPFB', 'sub_RVH', 'sub_RAO/RAE',
       'sub_IRBBB', 'sub_IVCD', 'sub_LMI', 'sub_AMI', 'sub__AVB', 'sub_ISCA',
       'sub_ISC_', 'sub_SEHYP', 'sub_ISCI', 'sub_CRBBB', 'sub_CLBBB',
       'sub_LAO/LAE', 'sub_ILBBB', 'sub_WPW', 'sub_PMI', 'strat_fold'],axis=1)
'''
testmeta = testmeta.drop(['MI','height','site','nurse','device', 'STTC', 'HYP', 'CD', 'sub_NORM', 'sub_IMI', 'sub_STTC',
       'sub_NST_', 'sub_LVH', 'sub_LAFB/LPFB', 'sub_RVH', 'sub_RAO/RAE',
       'sub_IRBBB', 'sub_IVCD', 'sub_LMI', 'sub_AMI', 'sub__AVB', 'sub_ISCA',
       'sub_ISC_', 'sub_SEHYP', 'sub_ISCI', 'sub_CRBBB', 'sub_CLBBB',
       'sub_LAO/LAE', 'sub_ILBBB', 'sub_WPW', 'sub_PMI', 'strat_fold'],axis=1)


In [4]:
sigdict = {}
for i in testmeta['ecg_id']:
    sigdict[i] = np.array(testsignal[testsignal['ecg_id']==i].drop(['ecg_id'],axis=1)).T

In [5]:
def hjorth(a):
    first_deriv = np.diff(a)
    second_deriv = np.diff(a,2)

    var_zero = np.mean(a ** 2)
    var_d1 = np.mean(first_deriv ** 2)
    var_d2 = np.mean(second_deriv ** 2)

    activity = var_zero
    morbidity = np.sqrt(var_d1 / var_zero)
    complexity = np.sqrt(var_d2 / var_d1) / morbidity

    return activity, morbidity, complexity

In [6]:

features = {}
for k,d in sigdict.items():
    features[k]={}
    for ch,row in enumerate(d):
        mean,std,kurt,skeww,minn,maxx,med,mode,q1,q3 = np.mean(row),np.std(row),kurtosis(row),skew(row),min(row),max(row),np.median(row),st.mode(row),np.quantile(row,.25),np.quantile(row,.75)
        iqr = q3-q1
        act,mob,comp = hjorth(row)
        petf,defa,heux = pfd(row),dfa(row),he(row)[0]
        perm,samp,shan = entropy.permutation_entropy(row,3,1),entropy.sample_entropy(row,2,0.2*np.std(row)),entropy.shannon_entropy(row)
        energy = 0
        for x in row:
            energy+=(x**2)
        features[k][ch] = [mean,std,kurt,skeww,minn,maxx,med,mode,q1,q3,iqr,act,mob,comp,petf,defa,heux,perm,samp,shan,energy]



In [15]:
testmeta = pd.concat([testmeta,testmeta,testmeta,testmeta,testmeta,testmeta,testmeta,testmeta,testmeta,testmeta,testmeta,testmeta],axis=0)
testmeta = testmeta.sort_values(by='ecg_id',ascending=True)

In [12]:
print(pd.DataFrame(features).shape)

(12, 2203)


In [16]:
testmeta

,ecg_id,age,sex,weight,NORM
0,9,55.0,0,70.0,1
0,9,55.0,0,70.0,1
0,9,55.0,0,70.0,1
0,9,55.0,0,70.0,1
0,9,55.0,0,70.0,1
...,...,...,...,...,...
2202,21826,82.0,1,NaN,0
2202,21826,82.0,1,NaN,0
2202,21826,82.0,1,NaN,0
2202,21826,82.0,1,NaN,0


In [36]:
result = {}
count = 0

for patient in features.keys():
    for channel in features[patient].keys():
        features[patient][channel][18] = features[patient][channel][18][0]
            
        result[count] = features[patient][channel]
        count += 1


In [58]:
xyz = pd.DataFrame(data=result,index=None).T.rename(columns={0:'mean',1:'std',2:'kurt',3:'skeww',4:'minn',5:'maxx',6:'med',7:'mode',8:'q1',9:'q3',10:'iqr',
                                                  11:'act',12:'mob',13:'comp',14:'petf',15:'defa',16:'heux',17:'perm',18:'samp',19:'shan',20:'energy'})

In [65]:
abc = xyz.reset_index(drop=True)
abc

,mean,std,kurt,skeww,minn,maxx,med,mode,q1,q3,...,act,mob,comp,petf,defa,heux,perm,samp,shan,energy
0,-0.002830,0.088689,1.537025,0.960132,-0.268,0.437,-0.0155,-0.048,-0.05900,0.03000,...,0.007874,0.542058,2.325225,1.021017,1.048476,0.213597,2.327949,1.995424,7.957670,7.873770
1,-0.004511,0.115144,4.296767,1.815879,-0.170,0.647,-0.0400,-0.096,-0.08000,0.03725,...,0.013278,0.466773,2.015152,1.019018,0.959661,0.177143,2.243332,1.731114,8.025056,13.278409
2,-0.001655,0.072583,16.299867,3.347911,-0.123,0.456,-0.0140,-0.011,-0.04400,0.02300,...,0.005271,0.623214,2.029970,1.023205,0.850505,0.269116,2.404263,1.701452,7.323528,5.271099
3,0.003746,0.096191,2.850263,-1.495380,-0.527,0.172,0.0300,0.043,-0.02925,0.06600,...,0.009267,0.475295,2.206709,1.020598,1.013287,0.194912,2.324463,1.856489,7.923268,9.266754
4,-0.000515,0.056977,6.815622,-1.447481,-0.332,0.157,0.0010,-0.027,-0.02800,0.03000,...,0.003247,0.670099,2.081799,1.024549,0.989849,0.239197,2.436917,1.955516,7.388032,3.246653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26431,0.001304,0.202849,18.739502,-3.845667,-1.332,0.487,0.0210,0.024,-0.02000,0.06100,...,0.041149,0.739199,1.528257,1.019915,0.845200,0.165620,2.295527,1.100608,7.834975,41.149356
26432,0.000714,0.255785,15.444687,-2.742884,-1.550,0.938,0.0060,0.004,-0.03325,0.06225,...,0.065426,0.842535,1.416392,1.018330,0.750103,0.136352,2.219317,1.005395,8.019224,65.426294
26433,0.000561,0.207170,15.982707,-2.713046,-1.264,0.817,0.0135,0.003,-0.02000,0.05400,...,0.042920,0.855730,1.397511,1.020703,0.720883,0.172475,2.322237,1.001742,7.771409,42.919849
26434,0.001638,0.142735,13.313774,0.071046,-0.711,0.791,0.0000,-0.010,-0.03500,0.03825,...,0.020376,0.932575,1.342337,1.020703,0.655043,0.179257,2.319767,1.316985,7.761233,20.375978


In [73]:
final=pd.concat([testmeta,xyz],axis=1,ignore_index=True).rename(columns={0:'ecg_id',1:'age',2:'sex',3:'weight',4:'label',5:'mean',6:'std',7:'kurt',8:'skeww',9:'minn',10:'maxx',11:'med',12:'mode'
                                                                   ,13:'q1',14:'q3',15:'iqr',
                                                  16:'act',17:'mob',18:'comp',19:'petf',20:'defa',21:'heux',22:'perm',23:'samp',24:'shan',25:'energy'}).dropna()
final


,ecg_id,age,sex,weight,label,mean,std,kurt,skeww,minn,...,act,mob,comp,petf,defa,heux,perm,samp,shan,energy
0,9,55.0,0,70.0,1,-0.002830,0.088689,1.537025,0.960132,-0.268,...,0.007874,0.542058,2.325225,1.021017,1.048476,0.213597,2.327949,1.995424,7.957670,7.873770
1,9,55.0,0,70.0,1,-0.004511,0.115144,4.296767,1.815879,-0.170,...,0.013278,0.466773,2.015152,1.019018,0.959661,0.177143,2.243332,1.731114,8.025056,13.278409
2,9,55.0,0,70.0,1,-0.001655,0.072583,16.299867,3.347911,-0.123,...,0.005271,0.623214,2.029970,1.023205,0.850505,0.269116,2.404263,1.701452,7.323528,5.271099
3,9,55.0,0,70.0,1,0.003746,0.096191,2.850263,-1.495380,-0.527,...,0.009267,0.475295,2.206709,1.020598,1.013287,0.194912,2.324463,1.856489,7.923268,9.266754
4,9,55.0,0,70.0,1,-0.000515,0.056977,6.815622,-1.447481,-0.332,...,0.003247,0.670099,2.081799,1.024549,0.989849,0.239197,2.436917,1.955516,7.388032,3.246653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25939,20943,52.0,0,80.0,1,-0.000547,0.240110,5.398331,-1.287738,-1.096,...,0.057653,0.564720,1.760359,1.015178,1.005151,0.068197,2.099683,1.228215,8.219720,57.652917
25940,20943,52.0,0,80.0,1,-0.004972,0.211330,8.163540,2.066106,-0.621,...,0.044685,0.811116,1.467803,1.018807,0.721918,0.049585,2.232742,1.293641,8.195116,44.685092
25941,20943,52.0,0,80.0,1,-0.009180,0.283520,19.491372,4.212337,-0.460,...,0.080468,0.824295,1.408722,1.019967,0.613092,0.064578,2.284104,1.147191,8.287253,80.467682
25942,20943,52.0,0,80.0,1,-0.003203,0.262060,20.450862,4.398518,-0.331,...,0.068685,0.792382,1.413730,1.019546,0.638446,0.066265,2.254366,1.058428,8.036850,68.685459


In [74]:
final.to_csv('extracted_features.csv',index=False)